## Goal: use of everything we've learned about APIs, databases, and Object-Oriented Programming to Extract, Transform, and Load (or ETL, for short) some data from a SQL database into a MongoDB Database.

Upon completion of this lab, each unique team in this dataset should have a record in the MongoDB instance containing the following information:

- 1. The name of the team
- 2. The total number of goals scored by the team during the 2011 season
- 3. The total number of wins the team earned during the 2011 season
- 4. A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly)
- 5. The team's win percentage on days where it was raining during games in the 2011 season.


Using Object-Oriented Programming is a requirement for this project--you must create at least 2 separate, well structured classes in your solution! Although it may seem easier to "just start coding", this is a classic beginner's mistake. Instead, think about separating out the different functionalities you'll need to reach your goal, and then build classes to handle each. For instance, at minimum, you'll need to:

- Query the SQL database
- Calculate summary statistics
- Get the weather data from the DarkSky API
- Load the data into MongoDB

We strongly recommend you consider creating separate classes for handling at least some of these of these tasks. Be sure to plan the inputs, outputs, and methods for each class before you begin coding!



## Inspect Database.sqlite

In [2]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [6]:
conn = sqlite3.connect('database.sqlite.db')
c = conn.cursor()

In [7]:
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(c.fetchall())

[('sqlite_sequence',), ('Matches',), ('Teams_in_Matches',), ('Teams',), ('Unique_Teams',), ('results',)]


In [8]:
tables = list(c.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall())
tables

[('sqlite_sequence',),
 ('Matches',),
 ('Teams_in_Matches',),
 ('Teams',),
 ('Unique_Teams',),
 ('results',)]

In [9]:
c.execute("SELECT name, type FROM sqlite_master;")
print(c.fetchall())

[('sqlite_sequence', 'table'), ('Matches', 'table'), ('sqlite_autoindex_Matches_1', 'index'), ('Teams_in_Matches', 'table'), ('Teams', 'table'), ('Unique_Teams', 'table'), ('FlatView_Chrono_TeamOrder_Reduced', 'view'), ('FlatView_Advanced', 'view'), ('FlatView', 'view'), ('results', 'table')]


In [10]:
c.execute('''select * from sqlite_sequence;''')
c.description


(('name', None, None, None, None, None, None),
 ('seq', None, None, None, None, None, None))

In [11]:
c.execute('''select * from sqlite_sequence;''')
df1 = pd.DataFrame(c.fetchall())
df1.columns = [x[0] for x in c.description]

In [61]:
df1

,name,seq
0,Matches,46774
1,Unique_Teams,128


In [12]:
c.execute('''select * from Teams_in_Matches;''')
df2 = pd.DataFrame(c.fetchall())
df2.columns = [x[0] for x in c.description]
print(len(df2))
df2.head()

49148


,Match_ID,Unique_Team_ID
0,1,26
1,1,46
2,2,26
3,2,42
4,3,26


Table: Teams_in_Matches

- Match_ID (int): Unique match ID
- Unique_Team_ID (int): Unique team ID (This table is used to easily retrieve each match a given team has played in)


In [13]:
c.execute('''select * from Teams;''')
df3 = pd.DataFrame(c.fetchall())
df3.columns = [x[0] for x in c.description]
print(len(df3))
df3.head()


468


,Season,TeamName,KaderHome,AvgAgeHome,ForeignPlayersHome,OverallMarketValueHome,AvgMarketValueHome,StadiumCapacity
0,2017,Bayern Munich,27,26,15,597950000,22150000,75000
1,2017,Dortmund,33,25,18,416730000,12630000,81359
2,2017,Leverkusen,31,24,15,222600000,7180000,30210
3,2017,RB Leipzig,30,23,15,180130000,6000000,42959
4,2017,Schalke 04,29,24,17,179550000,6190000,62271


Table: Teams

- Season (str): Football season for which the data is valid
- TeamName (str): Name of the team the data concerns
- KaderHome (str): Number of Players in the squad
- AvgAgeHome (str): Average age of players
- ForeignPlayersHome (str): Number of foreign players (non-German, non-English respectively) playing for the team
- OverallMarketValueHome (str): Overall market value of the team pre-season in EUR (based on data from transfermarkt.de)
- AvgMarketValueHome (str): Average market value (per player) of the team pre-season in EUR (based on data from transfermarkt.de)
- StadiumCapacity (str): Maximum stadium capacity of the team's home stadium


In [14]:
c.execute('''select * from Unique_Teams;''')
df4 = pd.DataFrame(c.fetchall())
df4.columns = [x[0] for x in c.description]
print(len(df4))
df4.head()


128


,TeamName,Unique_Team_ID
0,Bayern Munich,1
1,Dortmund,2
2,Leverkusen,3
3,RB Leipzig,4
4,Schalke 04,5


Table: Unique Teams

- TeamName (str): Name of a team
- Unique_Team_ID (int): Unique identifier for each team


In [15]:
c.execute('''select * from Matches;''').description


(('Match_ID', None, None, None, None, None, None),
 ('Div', None, None, None, None, None, None),
 ('Season', None, None, None, None, None, None),
 ('Date', None, None, None, None, None, None),
 ('HomeTeam', None, None, None, None, None, None),
 ('AwayTeam', None, None, None, None, None, None),
 ('FTHG', None, None, None, None, None, None),
 ('FTAG', None, None, None, None, None, None),
 ('FTR', None, None, None, None, None, None))

Table: Matches

- Match_ID (int): unique ID per match
- Div (str): identifies the division the match was played in (D1 = Bundesliga, D2 = Bundesliga 2, E0 = English Premier League)
- Season (int): Season the match took place in (usually covering the period of August till May of the following year)
- Date (str): Date of the match
- HomeTeam (str): Name of the home team
- AwayTeam (str): Name of the away team
- FTHG (int) (Full Time Home Goals): Number of goals scored by the home team
- FTAG (int) (Full Time Away Goals): Number of goals scored by the away team
- FTR (str) (Full Time Result): 3-way result of the match (H = Home Win, D = Draw, A = Away Win)

In [16]:
c.execute('''select * from Matches where season = 2011 order by date;''')
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
print(len(df))
df.head(100)

992


,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1133,D2,2011,2011-07-15,Cottbus,Dresden,2,1,H
1,1167,D2,2011,2011-07-15,Greuther Furth,Ein Frankfurt,2,3,A
2,1551,D2,2011,2011-07-15,Frankfurt FSV,Union Berlin,1,1,D
3,1550,D2,2011,2011-07-16,Erzgebirge Aue,Aachen,1,0,H
4,1678,D2,2011,2011-07-16,St Pauli,Ingolstadt,2,0,H
5,1146,D2,2011,2011-07-17,Karlsruhe,Duisburg,3,2,H
6,1442,D2,2011,2011-07-17,Braunschweig,Munich 1860,3,1,H
7,1602,D2,2011,2011-07-17,Hansa Rostock,Paderborn,1,2,A
8,1360,D2,2011,2011-07-18,Fortuna Dusseldorf,Bochum,2,0,H
9,1505,D2,2011,2011-07-22,Duisburg,Cottbus,1,2,A


## Date data for API

In [17]:
c.execute('''select Date from Matches where season = 2011 order by date;''')
dfd = pd.DataFrame(c.fetchall())
dfd.columns = [x[0] for x in c.description]
print(len(df))
dfd.head(100)

992


,Date
0,2011-07-15
1,2011-07-15
2,2011-07-15
3,2011-07-16
4,2011-07-16
5,2011-07-17
6,2011-07-17
7,2011-07-17
8,2011-07-18
9,2011-07-22


In [18]:
dfd.to_csv('date.csv')

In [182]:
df[df['HomeTeam']== 'Aachen']

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
15,1574,D2,2011,2011-07-24,Aachen,Braunschweig,0,2,A
46,1507,D2,2011,2011-08-13,Aachen,Cottbus,0,2,A
102,1702,D2,2011,2011-08-27,Aachen,Fortuna Dusseldorf,0,0,D
167,1529,D2,2011,2011-09-18,Aachen,Greuther Furth,0,0,D
206,1662,D2,2011,2011-09-30,Aachen,Frankfurt FSV,1,3,A
281,1682,D2,2011,2011-10-23,Aachen,Ingolstadt,3,1,H
335,1148,D2,2011,2011-11-06,Aachen,Duisburg,2,2,D
418,1446,D2,2011,2011-12-04,Aachen,Munich 1860,2,2,D
449,1595,D2,2011,2011-12-11,Aachen,Erzgebirge Aue,1,1,D
580,1465,D2,2011,2012-02-04,Aachen,St Pauli,2,1,H


In [181]:
df[df['HomeTeam']== 'Aachen']['FTHG'].sum()

15

## Identify the first and last day of the 2011 season

In [67]:
c.execute('''select date from Matches where season = 2011 order by date;''').fetchone()


('2011-07-15',)

In [68]:
c.execute('''select date from Matches where season = 2011 order by date desc;''').fetchone()


('2012-05-13',)

## Identify the set of teams played in 2011

In [163]:
hometeam = set(c.execute(
    '''select distinct hometeam from Matches where season = 2011 order by hometeam;''').fetchall())
print(len(hometeam))
hometeam

56


{('Aachen',),
 ('Arsenal',),
 ('Aston Villa',),
 ('Augsburg',),
 ('Bayern Munich',),
 ('Blackburn',),
 ('Bochum',),
 ('Bolton',),
 ('Braunschweig',),
 ('Chelsea',),
 ('Cottbus',),
 ('Dortmund',),
 ('Dresden',),
 ('Duisburg',),
 ('Ein Frankfurt',),
 ('Erzgebirge Aue',),
 ('Everton',),
 ('FC Koln',),
 ('Fortuna Dusseldorf',),
 ('Frankfurt FSV',),
 ('Freiburg',),
 ('Fulham',),
 ('Greuther Furth',),
 ('Hamburg',),
 ('Hannover',),
 ('Hansa Rostock',),
 ('Hertha',),
 ('Hoffenheim',),
 ('Ingolstadt',),
 ('Kaiserslautern',),
 ('Karlsruhe',),
 ('Leverkusen',),
 ('Liverpool',),
 ("M'gladbach",),
 ('Mainz',),
 ('Man City',),
 ('Man United',),
 ('Munich 1860',),
 ('Newcastle',),
 ('Norwich',),
 ('Nurnberg',),
 ('Paderborn',),
 ('QPR',),
 ('Schalke 04',),
 ('St Pauli',),
 ('Stoke',),
 ('Stuttgart',),
 ('Sunderland',),
 ('Swansea',),
 ('Tottenham',),
 ('Union Berlin',),
 ('Werder Bremen',),
 ('West Brom',),
 ('Wigan',),
 ('Wolfsburg',),
 ('Wolves',)}

In [164]:
awayteam = set(c.execute('''select distinct awayteam from Matches where season = 2011 order by awayteam;''').fetchall())
print(len(awayteam))
awayteam

56


{('Aachen',),
 ('Arsenal',),
 ('Aston Villa',),
 ('Augsburg',),
 ('Bayern Munich',),
 ('Blackburn',),
 ('Bochum',),
 ('Bolton',),
 ('Braunschweig',),
 ('Chelsea',),
 ('Cottbus',),
 ('Dortmund',),
 ('Dresden',),
 ('Duisburg',),
 ('Ein Frankfurt',),
 ('Erzgebirge Aue',),
 ('Everton',),
 ('FC Koln',),
 ('Fortuna Dusseldorf',),
 ('Frankfurt FSV',),
 ('Freiburg',),
 ('Fulham',),
 ('Greuther Furth',),
 ('Hamburg',),
 ('Hannover',),
 ('Hansa Rostock',),
 ('Hertha',),
 ('Hoffenheim',),
 ('Ingolstadt',),
 ('Kaiserslautern',),
 ('Karlsruhe',),
 ('Leverkusen',),
 ('Liverpool',),
 ("M'gladbach",),
 ('Mainz',),
 ('Man City',),
 ('Man United',),
 ('Munich 1860',),
 ('Newcastle',),
 ('Norwich',),
 ('Nurnberg',),
 ('Paderborn',),
 ('QPR',),
 ('Schalke 04',),
 ('St Pauli',),
 ('Stoke',),
 ('Stuttgart',),
 ('Sunderland',),
 ('Swansea',),
 ('Tottenham',),
 ('Union Berlin',),
 ('Werder Bremen',),
 ('West Brom',),
 ('Wigan',),
 ('Wolfsburg',),
 ('Wolves',)}

In [165]:
hometeam - awayteam

set()

### Clean special characters

In [166]:
team = c.execute(
    '''select distinct hometeam from Matches where season = 2011 order by hometeam;''').fetchall()
print(len(team))
team

56


[('Aachen',),
 ('Arsenal',),
 ('Aston Villa',),
 ('Augsburg',),
 ('Bayern Munich',),
 ('Blackburn',),
 ('Bochum',),
 ('Bolton',),
 ('Braunschweig',),
 ('Chelsea',),
 ('Cottbus',),
 ('Dortmund',),
 ('Dresden',),
 ('Duisburg',),
 ('Ein Frankfurt',),
 ('Erzgebirge Aue',),
 ('Everton',),
 ('FC Koln',),
 ('Fortuna Dusseldorf',),
 ('Frankfurt FSV',),
 ('Freiburg',),
 ('Fulham',),
 ('Greuther Furth',),
 ('Hamburg',),
 ('Hannover',),
 ('Hansa Rostock',),
 ('Hertha',),
 ('Hoffenheim',),
 ('Ingolstadt',),
 ('Kaiserslautern',),
 ('Karlsruhe',),
 ('Leverkusen',),
 ('Liverpool',),
 ("M'gladbach",),
 ('Mainz',),
 ('Man City',),
 ('Man United',),
 ('Munich 1860',),
 ('Newcastle',),
 ('Norwich',),
 ('Nurnberg',),
 ('Paderborn',),
 ('QPR',),
 ('Schalke 04',),
 ('St Pauli',),
 ('Stoke',),
 ('Stuttgart',),
 ('Sunderland',),
 ('Swansea',),
 ('Tottenham',),
 ('Union Berlin',),
 ('Werder Bremen',),
 ('West Brom',),
 ('Wigan',),
 ('Wolfsburg',),
 ('Wolves',)]

In [167]:
teams = []
for t in team:
    newt = t[0].translate({ord(i): ''''{}'''.format(i) for i in "'"})
    teams.append(newt)
teams
# hteam[0]

['Aachen',
 'Arsenal',
 'Aston Villa',
 'Augsburg',
 'Bayern Munich',
 'Blackburn',
 'Bochum',
 'Bolton',
 'Braunschweig',
 'Chelsea',
 'Cottbus',
 'Dortmund',
 'Dresden',
 'Duisburg',
 'Ein Frankfurt',
 'Erzgebirge Aue',
 'Everton',
 'FC Koln',
 'Fortuna Dusseldorf',
 'Frankfurt FSV',
 'Freiburg',
 'Fulham',
 'Greuther Furth',
 'Hamburg',
 'Hannover',
 'Hansa Rostock',
 'Hertha',
 'Hoffenheim',
 'Ingolstadt',
 'Kaiserslautern',
 'Karlsruhe',
 'Leverkusen',
 'Liverpool',
 "M''gladbach",
 'Mainz',
 'Man City',
 'Man United',
 'Munich 1860',
 'Newcastle',
 'Norwich',
 'Nurnberg',
 'Paderborn',
 'QPR',
 'Schalke 04',
 'St Pauli',
 'Stoke',
 'Stuttgart',
 'Sunderland',
 'Swansea',
 'Tottenham',
 'Union Berlin',
 'Werder Bremen',
 'West Brom',
 'Wigan',
 'Wolfsburg',
 'Wolves']

## #1&2 Sum team goals

In [157]:
c.execute('''select FTHG from Matches where season = 2011 and hometeam = 'Aachen';''').fetchall()

[(0,),
 (2,),
 (0,),
 (1,),
 (2,),
 (0,),
 (2,),
 (2,),
 (0,),
 (0,),
 (1,),
 (0,),
 (1,),
 (0,),
 (1,),
 (3,),
 (0,)]

In [158]:
c.execute('''select sum(FTHG) from Matches where season = 2011 and hometeam = 'Aachen';''').fetchall()

[(15,)]

In [175]:
hg=[]
for t in teams:
    q = '''select sum(FTHG) from Matches where season = 2011 and hometeam = '{}';'''.format(t)
    hg.append(c.execute(q).fetchall()[0][0])
hg

[15,
 39,
 20,
 20,
 49,
 26,
 23,
 23,
 21,
 41,
 18,
 44,
 30,
 23,
 38,
 19,
 28,
 20,
 35,
 20,
 24,
 36,
 46,
 19,
 31,
 20,
 19,
 21,
 28,
 12,
 23,
 28,
 24,
 29,
 27,
 55,
 52,
 35,
 29,
 28,
 22,
 28,
 24,
 47,
 36,
 25,
 36,
 26,
 27,
 39,
 30,
 31,
 21,
 22,
 29,
 19]

In [177]:
ag=[]
for t in teams:
    q = '''select sum(FTAG) from Matches where season = 2011 and awayteam = '{}';'''.format(t)
    ag.append(c.execute(q).fetchall()[0][0])
ag

[15,
 35,
 17,
 16,
 28,
 22,
 18,
 23,
 16,
 24,
 12,
 36,
 20,
 19,
 38,
 12,
 22,
 19,
 29,
 23,
 21,
 12,
 27,
 16,
 10,
 14,
 19,
 20,
 15,
 12,
 11,
 24,
 23,
 20,
 20,
 38,
 37,
 27,
 27,
 24,
 16,
 23,
 19,
 27,
 23,
 11,
 27,
 19,
 17,
 27,
 25,
 18,
 24,
 20,
 18,
 21]

## #3 Sum team wins

In [178]:
c.execute('''select count(FTR) from Matches where season = 2011 and hometeam = 'Aachen' and FTR = 'H';''').fetchall()

[(4,)]

In [183]:
hw=[]
for t in teams:
    q = '''select count(FTR) from Matches where season = 2011 and hometeam = '{}' and FTR = 'H';'''.format(t)
    hw.append(c.execute(q).fetchall()[0][0])
hw

[4,
 12,
 4,
 6,
 14,
 6,
 7,
 4,
 6,
 12,
 4,
 14,
 8,
 8,
 11,
 5,
 10,
 5,
 11,
 3,
 6,
 10,
 14,
 3,
 10,
 3,
 4,
 4,
 6,
 2,
 8,
 8,
 6,
 9,
 7,
 18,
 15,
 10,
 11,
 7,
 6,
 9,
 7,
 13,
 12,
 7,
 10,
 7,
 8,
 13,
 11,
 8,
 6,
 5,
 10,
 3]

In [184]:
aw=[]
for t in teams:
    q = '''select count(FTR) from Matches where season = 2011 and awayteam = '{}' and FTR = 'A';'''.format(t)
    aw.append(c.execute(q).fetchall()[0][0])
aw

[2,
 9,
 3,
 2,
 9,
 2,
 3,
 6,
 4,
 6,
 4,
 11,
 4,
 2,
 9,
 3,
 5,
 3,
 5,
 4,
 4,
 4,
 6,
 5,
 2,
 2,
 3,
 6,
 2,
 2,
 1,
 7,
 8,
 8,
 2,
 10,
 13,
 7,
 8,
 5,
 6,
 8,
 3,
 7,
 6,
 4,
 5,
 4,
 4,
 7,
 3,
 3,
 7,
 6,
 3,
 2]

## #4a Sum team losses

In [185]:
hl=[]
for t in teams:
    q = '''select count(FTR) from Matches where season = 2011 and hometeam = '{}' and FTR = 'A';'''.format(t)
    hl.append(c.execute(q).fetchall()[0][0])
hl

[7,
 3,
 8,
 4,
 2,
 12,
 7,
 11,
 3,
 4,
 5,
 1,
 4,
 7,
 1,
 5,
 6,
 8,
 1,
 4,
 5,
 4,
 2,
 7,
 0,
 7,
 10,
 4,
 3,
 10,
 6,
 5,
 4,
 1,
 7,
 0,
 2,
 5,
 3,
 6,
 7,
 2,
 7,
 3,
 2,
 4,
 4,
 5,
 4,
 3,
 4,
 5,
 10,
 7,
 5,
 13]

In [186]:
al=[]
for t in teams:
    q = '''select count(FTR) from Matches where season = 2011 and awayteam = '{}' and FTR = 'H';'''.format(t)
    al.append(c.execute(q).fetchall()[0][0])
al

[8,
 7,
 6,
 8,
 5,
 11,
 10,
 11,
 6,
 6,
 10,
 2,
 9,
 8,
 5,
 10,
 6,
 12,
 3,
 9,
 9,
 10,
 2,
 7,
 10,
 10,
 7,
 9,
 10,
 9,
 13,
 5,
 10,
 7,
 6,
 5,
 3,
 6,
 8,
 9,
 9,
 5,
 14,
 7,
 6,
 11,
 7,
 10,
 11,
 6,
 10,
 9,
 7,
 10,
 11,
 10]

## #4b Save results in new table and histogram

In [188]:
c.execute('''create table results (teamnames text, 
                                 homegoals integer, 
                                 awaygoals integer, 
                                 homewins integer, 
                                 awaywins integer, 
                                 homelosses integer,
                                 awaylosses integer
                                 )''')

In [189]:
c.description

In [ ]:
for i in teams:
    q = '''insert into results (teamnames) values '''

## Getting the Weather Data
Note that for this last calculation, you'll need to figure out if it was raining or not during the game. The database itself does not contain this information, but it does contain the date on which the game was played. For this, you'll need to use the DarkSky API to get the historical weather data for that day. Note that each game is played in a different location, and this information is not contained in our SQL database. However, the teams in this database are largely german, so go ahead and just use the weather in Berlin, Germany as a proxy for this information. If it was raining in Berlin on the day the game was played, count that as rain game--you do not need to try and figure out the actual weather at each game's location, because we don't have that information!

# NOTES

In [20]:
s = 'abc12321cba'

print(s.translate({ord(i): None for i in 'abc'}))

12321


In [87]:
tname = []
for t in tables:
    newt = t[0].translate({ord(i): None for i in '(),'})
    tname.append(newt)
tname

['sqlite_sequence', 'Matches', 'Teams_in_Matches', 'Teams', 'Unique_Teams']

In [39]:
for t in tables:
    print(t[0])
    c.execute(f"'''select * from {t[0]};'''")
    print(c.description)

sqlite_sequence


OperationalError: near "'''select * from sqlite_sequence;'''": syntax error